In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df= pd.read_csv("preprocessed_data.csv")
df.groupby('tag').describe()
labels=[]
for i in range(len(df)):
    if('obligation'in df['tag'][i]):
        labels.append('yes')
    else:
        labels.append('no')
rest_texts, test_texts, rest_labels, test_labels = train_test_split(df['sentence'], labels, test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)

print("Train size:", len(train_texts))
print("Dev size:", len(dev_texts))
print("Test size:", len(test_texts))

Train size: 766
Dev size: 86
Test size: 95


In [13]:
target_names = list(set(labels))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)

{'yes': 0, 'no': 1}


In [14]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('lr', LogisticRegression(multi_class="ovr", solver="lbfgs"))
])

parameters = {'lr__C': [0.1, 0.5, 1, 2, 5, 10, 100, 1000]}

best_classifier = GridSearchCV(pipeline, parameters, cv=5, verbose=1)
best_classifier.fit(train_texts, train_labels)
best_predictions = best_classifier.predict(test_texts)

baseline_accuracy = np.mean(best_predictions == test_labels)
print("Baseline accuracy:", baseline_accuracy)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Baseline accuracy: 0.8210526315789474


In [15]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
BERT_MODEL = "bert-base-uncased"
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

In [21]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = len(label2idx))
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
import logging
import numpy as np

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

MAX_SEQ_LENGTH=100

class BertInputItem(object):
    """An item with all the necessary attributes for finetuning BERT."""

    def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        

def convert_examples_to_inputs(example_texts, example_labels, label2idx, max_seq_length, tokenizer, verbose=0):
    """Loads a data file into a list of `InputBatch`s."""
    
    input_items = []
    examples = zip(example_texts, example_labels)
    for (ex_index, (text, label)) in enumerate(examples):

        # Create a list of token ids
        input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
        if len(input_ids) > max_seq_length:
            input_ids = input_ids[:max_seq_length]

        # All our tokens are in the first input segment (id 0).
        segment_ids = [0] * len(input_ids)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label2idx[label]

        input_items.append(
            BertInputItem(text=text,
                          input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))

        
    return input_items

train_features = convert_examples_to_inputs(train_texts, train_labels, label2idx, MAX_SEQ_LENGTH, tokenizer, verbose=0)
dev_features = convert_examples_to_inputs(dev_texts, dev_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_inputs(test_texts, test_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (710 > 512). Running this sequence through the model will result in indexing errors


In [23]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

def get_data_loader(features, max_seq_length, batch_size, shuffle=True): 

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader

BATCH_SIZE = 16

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
dev_dataloader = get_data_loader(dev_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

In [67]:
def evaluate(model, dataloader):
    model.eval()
    
    eval_loss = 0
    nb_eval_steps = 0
    predicted_labels, correct_labels = [], []

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        with torch.no_grad():
            tmp_eval_loss = model(input_ids, attention_mask=input_mask,
                                          token_type_ids=segment_ids, labels=label_ids).loss
            logits= model(input_ids, attention_mask=input_mask,
                                          token_type_ids=segment_ids, labels=label_ids).logits
            print(tmp_eval_loss)

        outputs = np.argmax(logits.to('cpu'), axis=1)
        label_ids = label_ids.to('cpu').numpy()
        
        predicted_labels += outputs
        correct_labels += list(label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    
    correct_labels = np.array(correct_labels)
    predicted_labels = np.array(predicted_labels)
        
    return eval_loss, correct_labels , predicted_labels


In [68]:
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup

GRADIENT_ACCUMULATION_STEPS = 1
NUM_TRAIN_EPOCHS = 20
LEARNING_RATE = 5e-5
WARMUP_PROPORTION = 0.1
MAX_GRAD_NORM = 5

num_train_steps = int(len(train_dataloader.dataset) / BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(WARMUP_PROPORTION * num_train_steps)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps, num_train_steps)

In [69]:
import torch
import os
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import classification_report, precision_recall_fscore_support

OUTPUT_DIR = "/tmp/"
MODEL_FILE_NAME = "pytorch_model.bin"
PATIENCE = 2

loss_history = []
no_improvement = 0
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        outputs = model(input_ids, attention_mask=input_mask, token_type_ids=segment_ids, labels=label_ids)
        loss = outputs[0]

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        tr_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)  
            
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            
    dev_loss, _, _ = evaluate(model, dev_dataloader)
    
    print("Loss history:", loss_history)
    print("Dev loss:", dev_loss)
    
    if len(loss_history) == 0 : #or dev_loss < min(loss_history):
        no_improvement = 0
        model_to_save = model.module if hasattr(model, 'module') else model
        output_model_file = os.path.join(OUTPUT_DIR, MODEL_FILE_NAME)
        torch.save(model_to_save.state_dict(), output_model_file)
    else:
        no_improvement += 1
    
    if no_improvement >= PATIENCE: 
        print("No improvement on development set. Finish training.")
        break
        
    
    loss_history.append(dev_loss)

Epoch:   0%|                                             | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_8456/1537822491.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm(train_dataloader, desc="Training iteration")):


Training iteration:   0%|          | 0/48 [00:00<?, ?it/s]

/tmp/ipykernel_8456/3885943829.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):


Evaluation iteration:   0%|          | 0/6 [00:00<?, ?it/s]

tensor(4.8959e-05, device='cuda:0')
tensor(0.0209, device='cuda:0')
tensor(0.7090, device='cuda:0')
tensor(0.7518, device='cuda:0')
tensor(8.2180e-06, device='cuda:0')
tensor(7.8479e-06, device='cuda:0')
Loss history: []
Dev loss: 0.24694426167661732


...

Training iteration:   0%|          | 0/48 [00:00<?, ?it/s]

Evaluation iteration:   0%|          | 0/6 [00:00<?, ?it/s]

tensor(1.9682e-05, device='cuda:0')
tensor(0.0063, device='cuda:0')
tensor(0.8231, device='cuda:0')
tensor(0.8588, device='cuda:0')
tensor(1.4082e-06, device='cuda:0')


Epoch:  10%|███▋                                 | 2/20 [00:14<02:07,  7.09s/it]

tensor(1.3312e-06, device='cuda:0')
Loss history: [0.24694426167661732, 0.2738210252045216]
Dev loss: 0.28136535717279304
No improvement on development set. Finish training.


In [70]:
model_state_dict = torch.load(os.path.join(OUTPUT_DIR, MODEL_FILE_NAME), map_location=lambda storage, loc: storage)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, state_dict=model_state_dict, num_labels = len(target_names))
model.to(device)

model.eval()

_, train_correct, train_predicted = evaluate(model, train_dataloader)
_, dev_correct, dev_predicted = evaluate(model, dev_dataloader)
_, test_correct, test_predicted = evaluate(model, test_dataloader)

print("Training performance:", precision_recall_fscore_support(train_correct, train_predicted, average="micro"))
print("Development performance:", precision_recall_fscore_support(dev_correct, dev_predicted, average="micro"))
print("Test performance:", precision_recall_fscore_support(test_correct, test_predicted, average="micro"))

bert_accuracy = np.mean(test_predicted == test_correct)

print(classification_report(test_correct, test_predicted, target_names=target_names))

/tmp/ipykernel_8456/3885943829.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):


Evaluation iteration:   0%|          | 0/48 [00:00<?, ?it/s]

tensor(7.7858e-06, device='cuda:0')
tensor(7.8082e-06, device='cuda:0')
tensor(7.8380e-06, device='cuda:0')
tensor(7.8678e-06, device='cuda:0')
tensor(8.2925e-06, device='cuda:0')
tensor(8.4638e-06, device='cuda:0')
tensor(7.5325e-06, device='cuda:0')
tensor(7.2643e-06, device='cuda:0')
tensor(8.7767e-06, device='cuda:0')
tensor(7.4431e-06, device='cuda:0')
tensor(7.9721e-06, device='cuda:0')
tensor(7.7933e-06, device='cuda:0')
tensor(7.2568e-06, device='cuda:0')
tensor(8.4191e-06, device='cuda:0')
tensor(9.0822e-06, device='cuda:0')
tensor(7.5996e-06, device='cuda:0')
tensor(8.1360e-06, device='cuda:0')
tensor(7.9348e-06, device='cuda:0')
tensor(8.1285e-06, device='cuda:0')
tensor(8.4191e-06, device='cuda:0')
tensor(7.6443e-06, device='cuda:0')
tensor(8.7171e-06, device='cuda:0')
tensor(8.3297e-06, device='cuda:0')
tensor(8.0838e-06, device='cuda:0')
tensor(8.3297e-06, device='cuda:0')
tensor(8.6650e-06, device='cuda:0')
tensor(7.7262e-06, device='cuda:0')
tensor(8.4415e-06, device='c

Evaluation iteration:   0%|          | 0/6 [00:00<?, ?it/s]

tensor(4.8959e-05, device='cuda:0')
tensor(0.0209, device='cuda:0')
tensor(0.7090, device='cuda:0')
tensor(0.7518, device='cuda:0')
tensor(8.2180e-06, device='cuda:0')
tensor(7.8479e-06, device='cuda:0')


Evaluation iteration:   0%|          | 0/6 [00:00<?, ?it/s]

tensor(1.4487, device='cuda:0')
tensor(2.2076, device='cuda:0')
tensor(2.1724e-05, device='cuda:0')
tensor(0.6833, device='cuda:0')
tensor(8.6054e-06, device='cuda:0')
tensor(7.5658e-06, device='cuda:0')
Training performance: (1.0, 1.0, 1.0, None)
Development performance: (0.9767441860465116, 0.9767441860465116, 0.9767441860465116, None)
Test performance: (0.9368421052631579, 0.9368421052631579, 0.9368421052631579, None)
              precision    recall  f1-score   support

         yes       0.92      0.92      0.92        37
          no       0.95      0.95      0.95        58

    accuracy                           0.94        95
   macro avg       0.93      0.93      0.93        95
weighted avg       0.94      0.94      0.94        95



In [64]:
model(input_ids, attention_mask=input_mask,
                                          token_type_ids=segment_ids, labels=label_ids)


SequenceClassifierOutput(loss=tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-4.9479,  4.2061],
        [ 4.2147, -4.0075],
        [-5.0396,  4.2019],
        [ 4.2294, -4.0396],
        [-5.0102,  4.1854],
        [-5.0054,  4.1997],
        [ 4.2364, -4.0481],
        [-4.9906,  4.2057],
        [ 4.2229, -4.0297],
        [ 4.2240, -4.0349],
        [-5.0509,  4.1822],
        [-4.9787,  4.1814],
        [-5.0388,  4.1931],
        [-5.0260,  4.1876]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)